# Start: Analysiere Daten

In [4]:
# import initial_data.csv
import pandas as pd
import numpy as np

from feasibility import is_feasible

# import data from csv file
def import_data():
    data = pd.read_csv('initial_data.csv')
    return data

data = import_data()

In [20]:
# put column names into a list
column_names = data.columns.values
print(column_names)
inputs = column_names[0:8]
print(inputs)
outputs = column_names[8:13]
print(outputs)

['Engine speed' 'Engine load' 'Railpressure' 'Air supply' 'Crank angle'
 'Intake pressure' 'Back pressure' 'Intake temperature' 'NOx' 'PM 1' 'CO2'
 'PM 2' 'Pressure cylinder' 'feasible' 'safe']
['Engine speed' 'Engine load' 'Railpressure' 'Air supply' 'Crank angle'
 'Intake pressure' 'Back pressure' 'Intake temperature']
['NOx' 'PM 1' 'CO2' 'PM 2' 'Pressure cylinder']


In [25]:
def get_input_data(row):
    x1, x2, x3, x4, x5, x6, x7, x8 = data.iloc[row, 0], data.iloc[row, 1], data.iloc[row, 2], data.iloc[row, 3], data.iloc[row, 4], data.iloc[row, 5], data.iloc[row, 6], data.iloc[row, 7]
    return x1, x2, x3, x4, x5, x6, x7, x8

def get_output_data(row):
    x1, x2, x3, x4, x5 = data.iloc[row, 8], data.iloc[row, 9], data.iloc[row, 10], data.iloc[row, 11], data.iloc[row, 12]
    return x1, x2, x3, x4, x5

def get_critical_output_data(i):
    x1, x2, x3 = data.iloc[i, 9], data.iloc[i, 11], data.iloc[i, 12]
    return x1, x2, x3

In [22]:
# show if data row is feasible via the first 8 columns and the function is_feasible
feasible = []
for i in range(len(data)):
    x1, x2, x3, x4, x5, x6, x7, x8 = get_input_data(i)
    # print(x1, x2, x3, x4, x5, x6, x7, x8)
    feasible.append(is_feasible(x1, x2, x3, x4, x5, x6, x7, x8))
data['feasible'] = feasible

In [26]:
# Check if outputs are in safe range
# PM 1 < 6, PM 2 < 16, Pressure cylinder < 160
safe = []
for i in range(len(data)):
    x9, x10, x11 = get_critical_output_data(i)
    if x9 < 6 and x10 < 16 and x11 < 160:
        safe.append(True)
    else:
        safe.append(False)
data['safe'] = safe

In [27]:
# show unsafe data (safe = False)
unsafe_data = data[data['safe'] == False]
print(unsafe_data)

    Engine speed  Engine load  Railpressure  Air supply  Crank angle  \
70        1800.0        35.66        1521.7    310.5600         5.69   
89        2000.0       139.54        2500.3   1008.3008         3.66   

    Intake pressure  Back pressure  Intake temperature         NOx      PM 1  \
70           1123.5         1236.1                60.9   66.869420  4.095885   
89           3129.9         3329.4                79.7  328.147428  4.017909   

           CO2       PM 2  Pressure cylinder  feasible   safe  
70   36.291467  16.283255          60.497930      True  False  
89  163.080520   4.692198         160.047375      True  False  


In [ ]:
# predict when to get safe data and when to get unsafe data
# PM 1 < 6, PM 2 < 16, Pressure cylinder < 160